# 3rd part: Testing the potential reasons for their causality for the miss predictions <br>


# 1. Testing the filled in NaNs for error causality
<br>
Now we are going to test if we ourselves are responsible for the false predictions by adding flawed information when we filled in the NaNs. We are going test this by adding information to the dataset about whether a loan contains a filled in former NaN value. We will achieve this by adding one extra column for each column that contained NaNs. If a row has a filled in NaN, we are going to add "Y" as a value in the newly added corresponding column otherwise we add "N". We then ordinally encode these features of the new columns, so the machine knows, that an filled NaN is inferior to a value that was present from the beginning. This way, if the random forest performs better, we will know that we added flawed information the way we filled in the NaNs. 

1.1 Importing and complementing required code from the previous part

In [1]:
import pandas as pd
import numpy as np

raw_df = pd.read_csv("https://github.com/TomMarq1/Credit_default_prediction_-ML-model-/raw/main/credit_risk_dataset.csv")

In [2]:
raw_df.drop_duplicates(inplace=True)

In [3]:
# add additional information regarding filled NaNs in "person_emp_length" and in "loan_int_rate"
raw_df["person_emp_length_filled_in"] = "N"
raw_df["loan_int_rate_filled_in"] = "N"

In [4]:
# add "Y" in "person_emp_length_filled_in" for every row that has NaN in "person_emp_lenght"
raw_df.loc[raw_df["person_emp_length"].isna(), ["person_emp_length_filled_in"]] = "Y"

In [5]:
# add "Y" in "loan_int_rate_filled_in" for every row that has NaN in "loan_int_rage"
raw_df.loc[raw_df["loan_int_rate"].isna(), ["loan_int_rate_filled_in"]] = "Y"

In [6]:
cleaned_df = raw_df.copy()
cleaned_df["person_emp_length"].fillna(0, inplace = True)

In [7]:
cleaned_df["loan_int_rate"].fillna(cleaned_df["loan_int_rate"].median(), inplace= True)

In [8]:
X = cleaned_df.drop(columns=["loan_status"])
y = cleaned_df["loan_status"]

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)

In [10]:
num_col = X_train.select_dtypes("number").columns.to_list()

In [11]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train_num_col_scaled = sc.fit_transform(X_train[num_col])
X_test_num_col_scaled = sc.transform(X_test[num_col])

In [12]:
cat_col = X_train.select_dtypes("object").columns.to_list()

In [13]:
cat_col

['person_home_ownership',
 'loan_intent',
 'loan_grade',
 'cb_person_default_on_file',
 'person_emp_length_filled_in',
 'loan_int_rate_filled_in']

In [14]:
non_ord_category_col = [cat_col[1]]

In [15]:
from sklearn.preprocessing import OneHotEncoder

ohenc = OneHotEncoder(sparse=False)
X_train_non_ord_cat_col_enc = ohenc.fit_transform(X_train[non_ord_category_col])
X_test_non_ord_cat_col_enc = ohenc.transform(X_test[non_ord_category_col])

In [16]:
ord_category_col = list((set(X_train.columns.to_list()) - set(num_col) - set(non_ord_category_col)))
ord_category_col

## for unkown reasons the order of the items in ord_category_col change every time the notebook is run. This can lead to an error in the cell 2 below. 
## to resolve the error rearrange the item order in cats_ord in the cell one below according to the item order of ord_category_col in this cell.

['loan_grade',
 'person_home_ownership',
 'person_emp_length_filled_in',
 'cb_person_default_on_file',
 'loan_int_rate_filled_in']

In [19]:
from sklearn.preprocessing import OrdinalEncoder

person_emp_length_filled_in_cats = ["Y", "N"]
cb_person_default_on_file_cats = ["Y", "N"]
loan_grade_cats =["G", "F", "E", "D", "C", "B", "A"]
loan_int_rate_filled_in_cats = ["Y", "N"]
person_home_ownership_cats = ["OTHER", "RENT", "MORTGAGE", "OWN"]

cats_ord = [loan_grade_cats, person_home_ownership_cats, person_emp_length_filled_in_cats, cb_person_default_on_file_cats, loan_int_rate_filled_in_cats]

In [20]:
# If an error occurs, look at comment 2 cells above to fix it.
ord_enc = OrdinalEncoder(categories= cats_ord)
X_train_ord_cat_enc = ord_enc.fit_transform(X_train[ord_category_col])
X_test_ord_cat_enc = ord_enc.transform(X_test[ord_category_col])


In [21]:
X_train_processed = pd.concat([pd.DataFrame(X_train_num_col_scaled, #dataframe with scaled numerical columns
                               columns = num_col),
                               pd.DataFrame(X_train_non_ord_cat_col_enc, #dataframe with encoded non ordinal categorical columns
                               columns = ohenc.get_feature_names_out()),
                               pd.DataFrame(X_train_ord_cat_enc, # dataframe with encoded ordinal categorical columns
                               columns = ord_category_col) 

                               
                               ], axis=1)

In [22]:
X_test_processed = pd.concat([pd.DataFrame(X_test_num_col_scaled, #dataframe with scaled numerical columns
                               columns = num_col),
                               pd.DataFrame(X_test_non_ord_cat_col_enc, #dataframe with encoded non ordinal categorical columns
                               columns = ohenc.get_feature_names_out()),
                               pd.DataFrame(X_test_ord_cat_enc, # dataframe with encoded ordinal categorical columns
                               columns = ord_category_col) 

                               
                               ], axis=1)

1.2 Train a random forest with the dataset which contains the added information with the hyperparameters we found with gridsearchCV in the previous part.

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [43]:
random_forest = RandomForestClassifier(max_depth=10, min_samples_leaf=50, min_samples_split=50, n_estimators=100, random_state=42)

In [44]:
random_forest.fit(X_train_processed, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=50, min_samples_split=50,
                       random_state=42)

In [45]:
accuracy_score(random_forest.predict(X_train_processed), y_train)

0.9272398748402451

In [46]:
accuracy_score(random_forest.predict(X_test_processed), y_test)

0.9274035989717224

1.3 Result
<br>
The random forest trained with added information performs slightly better with 92.74% accuracy compared to    92.72% without the added information. This increase in performance is in the margin of randomness so we can conclude that adding the information did not increase performance significantly and therby that we did not cause the false predictions by adding flawed informatin when filling in the NaNs. 

# 2. Testing "loan_grade > 3.5" as a source of the errors
<br>
Lastly we are going to test whether a value in "loan_grade" which is larger then 3.5 (=loan_grade C or better) contains flawed information because most of the miss predictions have such a value. To do so we are going to split up the dataset into 2 groups. One group contains loans with "loan_grade" <= 3.5 
(=loan_grade D or worse), the other group contains loans with "loan_grade" > 3.5 (=loan_grade C or better). 
<br>
If the value loan_grade > 3.5 is flawed, then having no loans of this kind in the group should result in a better prediction performance for this group compared to the prediction for the whole dataset.
<br>
Also if loan_grade > 3.5 is flawed , then having only such loans in one group and dropping these potentially flawed values by dropping the whole column "loan_grade", should result in higher performance compared to the same group without dropping the column with the potentially flawed values.


2.1 Testing group "loan_grade" <= 3.5 (loan grade D to G) 

In [20]:
# loan_grade D to G
import pandas as pd
raw_df = pd.read_csv("https://github.com/TomMarq1/Credit_default_prediction_-ML-model-/raw/main/credit_risk_dataset.csv")

In [21]:
cleaned_df = raw_df.copy()
cleaned_df["person_emp_length"].fillna(0, inplace = True)

In [22]:
# loan_grade_categories = ["A", "B", "C", "D", "E", "F", "G"]
cleaned_df["loan_int_rate"].fillna(cleaned_df["loan_int_rate"].median(), inplace= True)

In [23]:
X2 = cleaned_df.query("loan_grade == ['D', 'E', 'F', 'G']").drop(columns=["loan_status"])
y2 = cleaned_df.query("loan_grade == ['D', 'E', 'F', 'G']")["loan_status"]

In [24]:
from sklearn.model_selection import train_test_split
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.3, random_state=8)

In [25]:
num_col = X_train2.select_dtypes("number").columns.to_list()

In [26]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train2_num_col_scaled = sc.fit_transform(X_train2[num_col])
X_test2_num_col_scaled = sc.transform(X_test2[num_col])

In [27]:
cat_col = X_train2.select_dtypes("object").columns.to_list()

In [28]:
non_ord_category_col = [cat_col[1]]

In [29]:
from sklearn.preprocessing import OneHotEncoder
ohenc = OneHotEncoder(sparse=False)
X_train2_non_ord_cat_col_enc = ohenc.fit_transform(X_train2[non_ord_category_col])
X_test2_non_ord_cat_col_enc = ohenc.transform(X_test2[non_ord_category_col])

In [30]:
ord_category_col = list((set(X_train2.columns.to_list()) - set(num_col) - set(non_ord_category_col)))
ord_category_col
## for unkown reasons the order of the items in ord_category_col change every time the notebook is run. This can lead to an error in the cell 2 below. 
## to resolve the error rearrange the item order in cats_ord in the cell one below according to the item order of ord_category_col in this cell.

['loan_grade', 'person_home_ownership', 'cb_person_default_on_file']

In [33]:
cb_person_default_on_file_cats = ["Y", "N"]
loan_grade_cats =["G", "F", "E", "D", "C", "B", "A"]
person_home_ownership_cats = ["OTHER", "RENT", "MORTGAGE", "OWN"]

cats_ord = [loan_grade_cats, person_home_ownership_cats, cb_person_default_on_file_cats]


In [34]:
# if error occurs, see comment 2 cells above for fixing.
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder(categories= cats_ord)
X_train2_ord_cat_enc = ord_enc.fit_transform(X_train2[ord_category_col])
X_test2_ord_cat_enc = ord_enc.transform(X_test2[ord_category_col])


In [35]:
X_train2_processed = pd.concat([pd.DataFrame(X_train2_num_col_scaled, #dataframe with scaled numerical columns
                               columns = num_col),
                               pd.DataFrame(X_train2_non_ord_cat_col_enc, #dataframe with encoded non ordinal categorical columns
                               columns = ohenc.get_feature_names_out()),
                               pd.DataFrame(X_train2_ord_cat_enc, # dataframe with encoded ordinal categorical columns
                               columns = ord_category_col) 

                               
                               ], axis=1)

In [36]:
X_test2_processed = pd.concat([pd.DataFrame(X_test2_num_col_scaled, #dataframe with scaled numerical columns
                               columns = num_col),
                               pd.DataFrame(X_test2_non_ord_cat_col_enc, #dataframe with encoded non ordinal categorical columns
                               columns = ohenc.get_feature_names_out()),
                               pd.DataFrame(X_test2_ord_cat_enc, # dataframe with encoded ordinal categorical columns
                               columns = ord_category_col) 

                               
                               ], axis=1)

In [37]:
X_test2_processed

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,loan_intent_DEBTCONSOLIDATION,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,loan_grade,person_home_ownership,cb_person_default_on_file
0,0.620776,3.689454,0.958256,1.803236,-0.420820,-0.861115,0.243019,0.0,0.0,0.0,0.0,0.0,1.0,3.0,2.0,1.0
1,-0.621865,-0.771813,-1.010223,-0.787060,-0.650251,0.052253,-0.942658,0.0,0.0,0.0,0.0,1.0,0.0,3.0,1.0,0.0
2,-0.311204,-0.972771,-0.572783,-1.267734,-1.050536,-0.769778,-0.942658,0.0,1.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0
3,0.465446,-0.080518,0.739536,-0.399851,-0.664896,-0.495767,-0.231252,0.0,1.0,0.0,0.0,0.0,0.0,3.0,2.0,1.0
4,0.931437,0.715276,-0.135343,-0.413203,0.848373,-1.043788,0.954425,0.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1464,0.776107,-0.450280,0.302096,-0.933933,-0.347598,-0.861115,0.954425,1.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,1.0
1465,-0.466534,-0.369897,0.520816,-0.333091,-0.196271,-0.039083,-0.705523,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1.0,0.0
1466,-0.621865,-0.430184,-1.010223,-0.733652,0.848373,-0.587104,-0.705523,0.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0
1467,-0.777195,0.574605,0.520816,1.803236,1.624534,0.600274,-0.468387,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,1.0


2.2 Random forest with group d to g

In [40]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

random_forest2 = RandomForestClassifier(max_depth=10, min_samples_leaf=50, min_samples_split=50, n_estimators=100, random_state=42)

In [41]:
random_forest2.fit(X_train2_processed, y_train2)

RandomForestClassifier(max_depth=10, min_samples_leaf=50, min_samples_split=50,
                       random_state=42)

In [42]:
accuracy_score(random_forest2.predict(X_train2_processed), y_train2)

0.8931698774080561

In [43]:
accuracy_score(random_forest2.predict(X_test2_processed), y_test2)

0.8822328114363512

2.3 Result
<br>
The random forest performed worse with the group not containing loans having "loan_grade" > 3.5 then with the mixed original dataset. This can mean that having a value "loan_grade" > 3.5 is not flawed completly in every situation but instead provides useful information to learn how to predict the default of loans that have "loan_grade" <= 3.5. 


2.4 Testing "loan_grade" <= 3.5 (loan grade A to C)



In [98]:
X1 = cleaned_df.query("loan_grade == ['A', 'B', 'C']").drop(columns=["loan_status"])
y1 = cleaned_df.query("loan_grade == ['A', 'B', 'C']")["loan_status"]

In [99]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.3, random_state=8)

In [100]:
num_col = X_train2.select_dtypes("number").columns.to_list()

In [101]:
sc = StandardScaler()
X_train1_num_col_scaled = sc.fit_transform(X_train1[num_col])
X_test1_num_col_scaled = sc.transform(X_test1[num_col])

In [102]:
cat_col = X_train1.select_dtypes("object").columns.to_list()

In [103]:
non_ord_category_col = [cat_col[1]]

In [104]:
ohenc = OneHotEncoder(sparse=False)
X_train1_non_ord_cat_col_enc = ohenc.fit_transform(X_train1[non_ord_category_col])
X_test1_non_ord_cat_col_enc = ohenc.transform(X_test1[non_ord_category_col])

In [105]:
ord_category_col = list((set(X_train1.columns.to_list()) - set(num_col) - set(non_ord_category_col)))
ord_category_col

## for unkown reasons the order of the items in ord_category_col change every time the notebook is run. This can lead to an error in the cell 2 below. 
## to resolve the error rearrange the item order in cats_ord in the cell one below according to the item order of ord_category_col in this cell.

['loan_grade', 'person_home_ownership', 'cb_person_default_on_file']

In [106]:
cb_person_default_on_file_cats = ["Y", "N"]
loan_grade_cats =["G", "F", "E", "D", "C", "B", "A"]
person_home_ownership_cats = ["OTHER", "RENT", "MORTGAGE", "OWN"]

cats_ord = [loan_grade_cats, person_home_ownership_cats, cb_person_default_on_file_cats]


In [107]:
# if error occurs, see comment 2 cells above for fixing
ord_enc = OrdinalEncoder(categories= cats_ord)
X_train1_ord_cat_enc = ord_enc.fit_transform(X_train1[ord_category_col])
X_test1_ord_cat_enc = ord_enc.transform(X_test1[ord_category_col])

In [118]:
X_train1_processed = pd.concat([pd.DataFrame(X_train1_num_col_scaled, #dataframe with scaled numerical columns
                               columns = num_col),
                               pd.DataFrame(X_train1_non_ord_cat_col_enc, #dataframe with encoded non ordinal categorical columns
                               columns = ohenc.get_feature_names_out()),
                               pd.DataFrame(X_train1_ord_cat_enc, # dataframe with encoded ordinal categorical columns
                               columns = ord_category_col) 

                               
                               ], axis=1)

In [120]:
X_test1_processed = pd.concat([pd.DataFrame(X_test1_num_col_scaled, #dataframe with scaled numerical columns
                               columns = num_col),
                               pd.DataFrame(X_test1_non_ord_cat_col_enc, #dataframe with encoded non ordinal categorical columns
                               columns = ohenc.get_feature_names_out()),
                               pd.DataFrame(X_test1_ord_cat_enc, # dataframe with encoded ordinal categorical columns
                               columns = ord_category_col) 

                               
                               ], axis=1)

2.5 Random forest with group a to c

2.5.1 Random forest without column "loan_grade"

In [122]:
random_forest1 = RandomForestClassifier(max_depth=10, min_samples_leaf=50, min_samples_split=50, n_estimators=100, random_state=42)

In [123]:
random_forest1.fit(X_train1_processed.drop(columns=["loan_grade"]), y_train1)

RandomForestClassifier(max_depth=10, min_samples_leaf=50, min_samples_split=50,
                       random_state=42)

In [124]:
accuracy_score(random_forest1.predict(X_train1_processed.drop(columns=["loan_grade"])), y_train1)

0.9325593395252838

In [125]:
accuracy_score(random_forest1.predict(X_test1_processed.drop(columns=["loan_grade"])), y_test1)

0.9357091259330604

2.5.2 Random forest with column "loan_grade"

In [126]:
random_forest1 = RandomForestClassifier(max_depth=10, min_samples_leaf=50, min_samples_split=50, n_estimators=100, random_state=42)

In [127]:
random_forest1.fit(X_train1_processed, y_train1)

RandomForestClassifier(max_depth=10, min_samples_leaf=50, min_samples_split=50,
                       random_state=42)

In [128]:
accuracy_score(random_forest1.predict(X_train1_processed), y_train1)

0.9324045407636739

In [129]:
accuracy_score(random_forest1.predict(X_test1_processed), y_test1)

0.9353479412472911

2.6 Result
<br>
The random forest trained on the group only containing loans with "loan_grade" > 3.5 and in which the complete column "loan_grade" containing these potentially flawed values were dropped, performed almost identically to the random forest trained on the same group but keeping the column "loan_grade" (93.57% vs 93.53%).
<br>
This means that values "loan_grade" > 3.5 do not provide useful information for default prediction in a group consisting only of loans with "loan_grade" > 3.5. 

# 3. Combined results
<br>
On the one hand "loan_grade" > 3.5 as described above does not provide helpful information for default prediction for loans that have such a value in this range themselves. On the other hand values in that range seem to provide information that is useful to make predictions about other loans that have a value of =< 3.5 in "loan_grade".
<br>
<br>
The reason for this can unfortunately not be concluded from the dataset. But what can be said is that, if  loan grades from A to C do not provide information about the default possibility of loans with those values, then it is likely that the informations which were aggregated creating those values were not sufficient regarding predicting a default. By the disability to provide useful information about loan default for a certain group of loans, the "loan_grade" values A to C affect the dataquility of the whole dataset in a negative way.<br>
<br> 
In this case the method which is used to create those values, probably a scoring model of its own, should be revised by the bank to improve the possibility of default predictions for those kind of loans. This way the bank could have an overall more reliable default prediction which is a basis of pricing loans accordingly which itself is a corner stone of succesful banking. 

# 4. Conclusions

4.1 Main takeaways about machine learning from this task
<br>
<br>
My main takeaways about machine learning from this task are at first that there is basically an almost infinite number of iterations you can do to try to improve your model. All the possibilities of different machine learning models, different ways of filling NaNs, ways to rebalance a dataset, finding the right hyperparameters for a model and so on add up to a huge amount of possible combinations that all need to be tested to find the best possible outcome. Thereby building a machine learning model is strongly limited by the time you have to built it.
<br>
<br>
Furthermore a main takeaway is that even though you had a lot of time for building a machine learning model, you can not overcome issues with data quality (in this case "loan_grade" > 3.5). No matter how many and different methods you try to improve the models performance, you can not overcome a certain treshold which is set by the data quality.
<br>
<br>
Lastly the biggest takeaway for me personally was that the difficult and challenging thing about machine learning is not how to write the code but to explain to others what you have done, why, the findings and the conclusions of this complex matter in a way so that others can understand it.

4.2 Transforming the gained knowledge of this project into business insights <br>
<br>
Generally this project shows how important data quality is when collecting and creating new data by aggregation. Only if a certain data quality can be ensured, it makes economically sense to spent ressources for collecting data and making business decisions based on that data. If data quality is not sufficient, machine learning models as well as deciders will make incorrect decisions based on this data without anyone even noticing until it´s too late and the damage is already done.<br>
<br>
In the instant case, as already mentioned above, the bank should revise the method which is used to create loan grade values from A to C, probably a scoring model of its own, to improve the possibility of default predictions for those kind of loans. This way the bank can have an overall more reliable default prediction which is a basis of pricing loans accordingly which itself is a corner stone of succesful banking. 